In [1]:
#
# Author: Yu Cao
#
#
#
# 2-layered neural network for autonomous car trajectory predictions

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy as np
import pickle
from glob import glob

import itertools
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import time

In [2]:
import pickle

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


In [3]:
new_path = "./new_train/new_train"
val_dataset  = ArgoverseDataset(data_path=new_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train_in, train_label = next(iter(val_loader))
train_in.size()

device(type='cuda')

In [37]:
batch_sz = 2000

In [38]:
def my_collate(batch):
    inp = []
    for scene in batch:
        a = []
        agent_id = scene['agent_id']
        agent_idx = np.where(scene['track_id'] == agent_id)[0][0]
        p_input = scene['p_in'][agent_idx]
        inp.append([item for sublist in p_input for item in sublist])
    out = []
    for scene in batch:
        agent_id = scene['agent_id']
        agent_idx = np.where(scene['track_id'] == agent_id)[0][0]
        p_out = scene['p_out'][agent_idx]
        out.append([item for sublist in p_out for item in sublist])
    
    inp = torch.FloatTensor(inp)
    # size: batch size x 38 (linear p_in)
    
    out = torch.FloatTensor(out)
    # size: batch size x 60 (linear p_out)
    
    return [inp, out]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [40]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hid1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.hid2 = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hid1(x))
        x = self.hid2(x)            # linear output length 60
        return x

# net = torch.nn.Sequential(
#         torch.nn.Linear(38, 200),
#         torch.nn.LeakyReLU(),
#         torch.nn.Linear(200, 100),
#         torch.nn.LeakyReLU(),
#         torch.nn.Linear(100, 60),
#     )

In [41]:
net = Net(n_feature=38, n_hidden=2000, n_output=60).to(device)
# print(net)  # network architecture
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss() 

In [42]:
import time
start_time = time.time()
for epoch in range(3):  # loop over the dataset multiple time
    running_loss = 0.0
    for i, data in enumerate(val_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print(outputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            #print(outputs[0])
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0
print("--- %s seconds ---" % (time.time() - start_time))

[1,    50] loss: 5302211.685
[1,   100] loss: 3747289.255


KeyboardInterrupt: 

In [ ]:
MODEL_PATH = 'model.pth' # save model
torch.save(net, MODEL_PATH)

In [25]:
net = torch.load(MODEL_PATH)

In [26]:
net

Net(
  (hid1): Linear(in_features=38, out_features=100, bias=True)
  (hid2): Linear(in_features=100, out_features=60, bias=True)
)

In [11]:
# get input tensors for test data
def test_collate(batch):
    inp = []
    for scene in batch:
        a = []
        agent_id = scene['agent_id']
        agent_idx = np.where(scene['track_id'] == agent_id)[0][0]
        p_input = scene['p_in'][agent_idx]
        inp.append([item for sublist in p_input for item in sublist])
    
    inp = torch.FloatTensor(inp)
    return inp

In [12]:
test_dataset  = ArgoverseDataset(data_path="./new_val_in/new_val_in")
test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = False, collate_fn=test_collate, num_workers=0)
test_in = next(iter(test_loader))
submission = pd.read_csv('sample_submission.csv')

In [31]:
predictions = []
for i, data in enumerate(test_loader, 0):
    data = data.to(device)
    pred = net(data)
    predictions.append(pred)

In [32]:
# convert output tensors to list of lists
predictions_lst = []
for batch in predictions:
    for scene in batch:
        predictions_lst.append(scene.tolist())
scene_id = submission['ID']
scene_id = scene_id.tolist()
for i in range(len(predictions_lst)):
    predictions_lst[i].insert(0, scene_id[i])

In [33]:
# save as csv
first_submission = pd.DataFrame(predictions_lst, columns = submission.columns)
first_submission.to_csv('fc_submission3', index=False)